In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import keras.layers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import os
from os import listdir
import PIL
from PIL import Image
from PIL import ImageOps
from imutils import paths

import numpy as np

In [3]:
image_size = (224, 224)
dropout = 0.2
batch_size = 32
lr = 1e-4
epochs = 20

In [4]:
maskoff_paths = list(paths.list_images('faces/maskoff/'))
maskon_paths = list(paths.list_images('faces/maskon/'))

data = []
labels = []

# maskoff
for maskoff_path in maskoff_paths:
    label = 'maskoff'
    image = load_img(maskoff_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

# maskon
for maskon_path in maskon_paths:
    label = 'maskon'
    image = load_img(maskon_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

In [5]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [6]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = keras.utils.to_categorical(labels)

In [7]:
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [8]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=image_size + (3,))
head_model = base_model.output
head_model = keras.layers.AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = keras.layers.Flatten(name="flatten")(head_model)
head_model = keras.layers.Dense(128, activation="relu")(head_model)
head_model = keras.layers.Dropout(0.5)(head_model)
head_model = keras.layers.Dense(2, activation="softmax")(head_model)

model = keras.models.Model(inputs=base_model.input, outputs=head_model)
for layer in base_model.layers:
	layer.trainable = False

In [9]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
train_generator = aug.flow(
    train_x, train_y,
    batch_size = batch_size
)

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint("saved_models/model_1/save_at_{epoch}.keras")
]

In [11]:
opt = keras.optimizers.Adam(lr =lr, decay = lr / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_x) // batch_size,
	validation_data=(test_x, test_y),
	validation_steps=len(test_x) // batch_size,
	epochs=epochs,
    callbacks=callbacks)

Epoch 1/20
47/47 [==============================] - 62s 1s/step - loss: 0.6943 - accuracy: 0.6477 - val_loss: 0.4843 - val_accuracy: 0.7857
Epoch 2/20
47/47 [==============================] - 51s 1s/step - loss: 0.4875 - accuracy: 0.7805 - val_loss: 0.3617 - val_accuracy: 0.8651
Epoch 3/20
47/47 [==============================] - 51s 1s/step - loss: 0.3891 - accuracy: 0.8462 - val_loss: 0.3119 - val_accuracy: 0.8862
Epoch 4/20
47/47 [==============================] - 57s 1s/step - loss: 0.3490 - accuracy: 0.8706 - val_loss: 0.2669 - val_accuracy: 0.9074
Epoch 5/20
47/47 [==============================] - 55s 1s/step - loss: 0.3134 - accuracy: 0.8821 - val_loss: 0.2458 - val_accuracy: 0.9180
Epoch 6/20
47/47 [==============================] - 53s 1s/step - loss: 0.2954 - accuracy: 0.8950 - val_loss: 0.2227 - val_accuracy: 0.9312
Epoch 7/20
47/47 [==============================] - 52s 1s/step - loss: 0.2629 - accuracy: 0.9004 - val_loss: 0.2156 - val_accuracy: 0.9286
Epoch 8/20
47/47 [==